# Instant Gratification

## Your first Frontier LLM Project!

Let's build a useful LLM solution - in a matter of minutes.

By the end of this course, you will have built an autonomous Agentic AI solution with 7 agents that collaborate to solve a business problem. All in good time! We will start with something smaller...

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, you should have completed the setup for [PC](../SETUP-PC.md) or [Mac](../SETUP-mac.md) and you hopefully launched this jupyter lab from within the project root directory, with your environment activated.

## If you're new to Jupyter Lab

Welcome to the wonderful world of Data Science experimentation! Once you've used Jupyter Lab, you'll wonder how you ever lived without it. Simply click in each "cell" with code in it, such as the cell immediately below this text, and hit Shift+Return to execute that cell. As you wish, you can add a cell with the + button in the toolbar, and print values of variables, or try out variations.  

I've written a notebook called [Guide to Jupyter](Guide%20to%20Jupyter.ipynb) to help you get more familiar with Jupyter Labs, including adding Markdown comments, using `!` to run shell commands, and `tqdm` to show progress.

## If you'd prefer to work in IDEs

If you're more comfortable in IDEs like VSCode or Pycharm, they both work great with these lab notebooks too.  
If you'd prefer to work in VSCode, [here](https://chatgpt.com/share/676f2e19-c228-8012-9911-6ca42f8ed766) are instructions from an AI friend on how to configure it for the course.

## If you'd like to brush up your Python

I've added a notebook called [Intermediate Python](Intermediate%20Python.ipynb) to get you up to speed. But you should give it a miss if you already have a good idea what this code does:    
`yield from {book.get("author") for book in books if book.get("author")}`

## I am here to help

If you have any problems at all, please do reach out.  
I'm available through the platform, or at ed@edwarddonner.com, or at https://www.linkedin.com/in/eddonner/ if you'd like to connect (and I love connecting!)

## More troubleshooting

Please see the [troubleshooting](troubleshooting.ipynb) notebook in this folder to diagnose and fix common problems. At the very end of it is a diagnostics script with some useful debug info.

## If this is old hat!

If you're already comfortable with today's material, please hang in there; you can move swiftly through the first few labs - we will get much more in depth as the weeks progress.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Please read - important note</h2>
            <span style="color:#900;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you do this with me, either at the same time, or (perhaps better) right afterwards. Add print statements to understand what's going on, and then come up with your own variations. If you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...</span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business value of these exercises</h2>
            <span style="color:#181;">A final thought. While I've designed these notebooks to be educational, I've also tried to make them enjoyable. We'll do fun things like have LLMs tell jokes and argue with each other. But fundamentally, my goal is to teach skills you can apply in business. I'll explain business implications as we go, and it's worth keeping this in mind: as you build experience with models and techniques, think of ways you could put this into action at work today. Please do contact me if you'd like to discuss more or if you have ideas to bounce off me.</span>
        </td>
    </tr>
</table>

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

## Troubleshooting if you have problems:

Head over to the [troubleshooting](troubleshooting.ipynb) notebook in this folder for step by step code to identify the root cause and fix it!

If you make a change, try restarting the "Kernel" (the python process sitting behind this notebook) by Kernel menu >> Restart Kernel and Clear Outputs of All Cells. Then try this notebook again, starting at the top.

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [2]:
from dotenv import load_dotenv
import os

current_dir = os.getcwd()

# Specify the path to the .env file in the parent folder
dotenv_path = os.path.join(current_dir, '..', '.env')

# Load the .env file
load_dotenv(dotenv_path)

# Access environment variables
my_variable = os.getenv('OPENAI_API_KEY')
print(my_variable)

sk-q0M/eqRGQBixrWXoHoxZmW7/H/jBKpgbwAgrICF+i2RQVVZHisIHOcPG7nng99TdfjjboRnX32Zlb6WVpTvVfUGWQceYMzk8Su4ZvZYSGQE=


In [3]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook


In [4]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

In [5]:
client = OpenAI(
    api_key=api_key,
    base_url="https://router.requesty.ai/v1"
)

try:
    # Make your API call
    response = client.chat.completions.create(
        model="deepseek/deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Hello! How can you help me today?"}
        ]
    )
    
    # Print the assistant's response
    print("Assistant:", response.choices[0].message.content)

except openai.APIError as e:
    print(f"OpenAI API error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

Assistant: Hello! I can assist you with a wide range of tasks, including answering questions, providing explanations, helping with research, offering advice, generating creative content, and much more. What do you need help with today? 😊


# Let's make a quick call to a Frontier model to get started, as a preview!

In [6]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = client.chat.completions.create(model="deepseek/deepseek-chat", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! 👋 Welcome to the world of chatting with me—I'm so glad you're here! How can I help you today? Whether it's a question, a fun fact, or just a chat, I'm all ears (or, well, algorithms). 😊


## OK onwards with our first project

In [7]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [8]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [9]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [10]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [11]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the might GPT (yet!)

In [12]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [13]:
# To give you a preview -- calling OpenAI with system and user messages:

response = client.chat.completions.create(model="deepseek/deepseek-chat", messages=messages)
print(response.choices[0].message.content)




Oh, come on, really? It's *4*. But hey, if you're feeling adventurous, maybe it's 22 in some alternate universe where math is just a suggestion. 😏


In [ ]:
print("this is a test message ")




this is a test 


## And now let's build useful messages for GPT-4o-mini, using a function

In [14]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [15]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, posi

## Time to bring it together - the API for OpenAI is very simple!

In [18]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = client.chat.completions.create(
        model = "deepseek/deepseek-chat",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [19]:
summarize("https://edwarddonner.com")

'```markdown\n# Summary of Edward Donner\'s Website\n\nEdward Donner\'s website serves as a personal and professional hub, showcasing his interests, projects, and career achievements. Here are the key highlights:\n\n## About Edward Donner\n- **Interests**: Coding, experimenting with LLMs, DJing, electronic music production, and engaging with tech communities like Hacker News.\n- **Professional Role**: Co-founder and CTO of **Nebula.io**, an AI-driven platform focused on helping individuals discover their potential and assisting recruiters in talent management.\n- **Background**: Former founder and CEO of AI startup **untapt**, which was acquired in 2021. He has expertise in proprietary LLMs and patented matching models.\n\n## Projects\n- **Outsmart**: An arena where LLMs compete in diplomacy and strategy, reflecting his interest in AI experimentation.\n\n## Announcements\n- **December 21, 2024**: Welcoming "SuperDataScientists" to the platform.\n- **November 13, 2024**: Sharing resourc

In [20]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [21]:
display_summary("https://edwarddonner.com")

```markdown
# Summary of Edward Donner's Website

## About Edward Donner
Edward Donner is a co-founder and CTO of **Nebula.io**, a company applying AI to help people discover their potential and pursue their passions. He is also the former founder and CEO of **untapt**, an AI startup acquired in 2021. Edward enjoys coding, experimenting with LLMs, DJing, and electronic music production. His work focuses on proprietary LLMs verticalized for talent, with a patented matching model and a successful platform.

## Key Announcements
- **December 21, 2024**: Welcome message to **SuperDataScientists**.
- **November 13, 2024**: Resources for **Mastering AI and LLM Engineering**.
- **October 16, 2024**: Guide on transitioning **From Software Engineer to AI Data Scientist**.
- **August 6, 2024**: Launch of **Outsmart LLM Arena**, a platform where LLMs compete in diplomacy and strategy.

## Projects
- **Outsmart**: An arena where LLMs compete in a battle of diplomacy and deviousness.

## Contact
- Email: ed [at] edwarddonner [dot] com
- Social Media: LinkedIn, Twitter, Facebook
- Newsletter subscription available on the website.
```

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [24]:
display_summary("https://cnn.com")

```markdown
# Summary of CNN Website

The CNN website provides a comprehensive platform for breaking news, latest updates, and videos across various categories. Below is a summary of the key sections and recent news highlights:

## Key Sections:
- **US**: Crime, Justice, and Politics.
- **World**: Coverage of Africa, Americas, Asia, Europe, and the Middle East.
- **Politics**: SCOTUS, Congress, and 2024 Elections.
- **Business**: Tech, Media, and Markets.
- **Health**: Fitness, Food, Sleep, and Relationships.
- **Entertainment**: Movies, Television, and Celebrity news.
- **Style**: Arts, Design, Fashion, and Beauty.
- **Travel**: Destinations, Food & Drink, and Stay.
- **Sports**: Pro Football, Basketball, Soccer, and Olympics.
- **Science**: Space, Life, and Climate.
- **Weather**: Updates and forecasts.
- **Ukraine-Russia War**: Ongoing coverage.
- **Israel-Hamas War**: Latest developments.

## Recent News Highlights:
- **Politics**: Treasury pick Scott Bessent's confirmation hearing and Trump's administration ethics questions.
- **Israel-Hamas War**: Israel delays key vote on truce; Hamas commits to the deal.
- **Health**: Young women are almost twice as likely to be diagnosed with cancer as young men.
- **Science**: Blue Origin launches New Glenn rocket, a competitor to SpaceX.
- **Sports**: 2024 NFL divisional round updates and Super Bowl streaming details.
- **Entertainment**: Drake sues UMG Recordings over defamation claims related to Kendrick Lamar’s song.
- **Business**: Outgoing FCC chair rejects high-profile TV petitions.
- **Travel**: Restaurant charges $120 to add pineapple on pizza.

## Additional Features:
- **CNN Underscored**: Product reviews and deals.
- **Live TV and Podcasts**: CNN Headlines, CNN 5 Things, and various audio podcasts.
- **Games**: Crossword, Sudoku, and other puzzles.

## Special Reports:
- **LA Wildfires**: Coverage of the ongoing wildfires and their impact.
- **Climate**: Solutions and weather-related updates.
- **Analysis**: Insights into political and social issues, including Trump’s potential second term.

## Interactive Elements:
- **Ad Feedback**: Users can provide feedback on ads and report technical issues.
- **Sign In**: Personalized account features for users to follow topics and access settings.

CNN continues to be a leading source for global news, offering a mix of live updates, in-depth analysis, and multimedia content.
```

In [25]:
display_summary("https://anthropic.com")

# Summary of Anthropic's Website

Anthropic is an AI safety and research company based in San Francisco, focused on developing reliable and beneficial AI systems. Their flagship product, **Claude**, is an AI model designed with safety at its core. The website highlights their latest AI model, **Claude 3.5 Sonnet**, which is now available and described as their most intelligent model to date. Additionally, they offer **Claude 3.5 Haiku**, another advanced AI model.

## Key Announcements:
1. **Claude 3.5 Sonnet Release** (October 22, 2024): Introduced as their most intelligent AI model, alongside **Claude 3.5 Haiku**.
2. **Claude for Enterprise** (September 4, 2024): A product tailored for enterprise use.
3. **Core Views on AI Safety** (March 8, 2023): Anthropic shared their perspectives on AI safety, addressing the "when, why, what, and how" of ensuring AI systems are safe and beneficial.
4. **Constitutional AI: Harmlessness from AI Feedback** (December 15, 2022): Research focused on creating AI systems that are inherently harmless through feedback mechanisms.

## Products and Services:
- **Claude API**: Enables developers to build AI-powered applications and custom experiences.
- **Enterprise Solutions**: Tailored AI solutions for businesses.
- **Research**: Anthropic emphasizes alignment and safety in AI development, with ongoing research in areas like Constitutional AI.

## Careers:
Anthropic is hiring for various roles, seeking individuals with expertise in machine learning, physics, policy, and product development.

For more details, visit [Claude.ai](https://claude.ai).

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [32]:
# Step 1: Create your prompts

system_prompt = "something here"
user_prompt = """
    Try to make sense of the input message lists
"""

# Step 2: Make the messages list

messages_list = [
    "all-purpose flour", "bread flour", "00 flour", "whole wheat flour",
    "active dry yeast", "instant yeast", "fresh yeast", "warm water", "salt", "sugar", "olive oil",
    "canned crushed tomatoes", "canned whole peeled tomatoes", "tomato paste", "fresh tomatoes",
    "garlic", "onion", "oregano", "basil", "pepper", "red pepper flakes",
    "pesto", "white sauce", "bbq sauce", "alfredo sauce",
    "mozzarella", "provolone", "parmesan", "ricotta", "fontina", "gorgonzola", "cheddar",
    "pepperoni", "italian sausage", "chorizo", "ham", "bacon", "ground beef", "chicken", "meatballs",
    "mushrooms", "onions", "bell peppers", "olives", "tomatoes", "spinach", "artichoke hearts",
    "jalapeños", "pineapple", "broccoli", "zucchini", "eggplant",
    "fresh basil", "dried oregano", "fresh oregano", "minced garlic", "roasted garlic", "rosemary", "thyme",
    "balsamic glaze"
]

# Step 3: Call OpenAI

response = client.chat.completions.create(
    model="deepseek/deepseek-chat",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt + "\n".join(messages_list)},
        {"role": "user", "content": "tell me more about this"}
    ]
)

# Step 4: print the result

print(response.choices[0].message.content)

This list appears to be a comprehensive collection of ingredients commonly used in cooking, particularly in Italian and Mediterranean cuisines, with a focus on pizza, pasta, and other baked or saucy dishes. Here's a breakdown of the categories and their significance:

### **1. Flours**
   - **All-purpose flour**: A versatile flour used for a variety of baked goods.
   - **Bread flour**: Higher protein content, ideal for bread-making due to its gluten strength.
   - **00 flour**: A finely milled Italian flour, perfect for pizza dough and pasta.
   - **Whole wheat flour**: Adds a nutty flavor and extra fiber to baked goods.

### **2. Yeasts**
   - **Active dry yeast**: Requires activation in warm water before use.
   - **Instant yeast**: Can be mixed directly into dry ingredients.
   - **Fresh yeast**: Also known as cake yeast, used in professional baking.

### **3. Liquids and Seasonings**
   - **Warm water**: Used to activate yeast and hydrate dough.
   - **Salt**: Enhances flavor and 

In [31]:
response

ChatCompletion(id='5ffe0a0e-8ce0-4312-b8b5-143ca0380640', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="This list appears to be a comprehensive collection of ingredients commonly used in cooking, particularly in Italian and Mediterranean cuisines, with a focus on dishes like pizza, pasta, and sauces. Here's a breakdown of the categories and their significance:\n\n### **Flours**\n- **All-purpose flour**: A versatile flour used for a wide range of baked goods.\n- **Bread flour**: Higher protein content, ideal for bread-making due to its gluten strength.\n- **00 flour**: A finely milled Italian flour, perfect for pizza dough and pasta.\n- **Whole wheat flour**: Adds a nutty flavor and more fiber to baked goods.\n\n### **Yeast**\n- **Active dry yeast**: Needs to be activated in warm water before use.\n- **Instant yeast**: Can be mixed directly into dry ingredients.\n- **Fresh yeast**: Also known as cake yeast, often used in professiona

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

Here are good instructions courtesy of an AI friend:  
https://chatgpt.com/share/677a9cb5-c64c-8012-99e0-e06e88afd293